In [1]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
import time
from sklearn.metrics import accuracy_score, confusion_matrix
import gensim
from sklearn.svm import LinearSVC
import random

In [2]:
imdb = "E:/Momo/Datasets/imdbreviews/IMDB Dataset.csv"

df = pd.read_csv(imdb)
print(df.shape)
df.head()

(50000, 2)


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [3]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [4]:
def clean_text(text):
    text=str(text).lower()
    text=re.sub('\[.*?\]', '', text)
    text=re.sub('https?://\S+|www\.\S+', '', text)
    text=re.sub('<.*?>+', '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = gensim.parsing.remove_stopwords(text)
    return text

df['review']=df['review'].apply(clean_text)
df.head()

,review,sentiment
0,reviewers mentioned watching oz episode you'll...,positive
1,wonderful little production. filming technique...,positive
2,thought wonderful way spend time hot summer we...,positive
3,basically there's family little boy (jake) thi...,negative
4,"petter mattei's ""love time money"" visually stu...",positive


In [5]:
# Split features and labels
X = df["review"]
y = df["sentiment"]

# Split train and test set
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=0)

print(X_train.shape, X_test.shape)

(35000,) (15000,)


In [6]:
# Create feature vectors
vectorizer = CountVectorizer()
train_vectors = vectorizer.fit_transform(X_train)
test_vectors = vectorizer.transform(X_test)

In [7]:
def fitness(tol, C, intercept_scaling):
    model = LinearSVC(tol=tol, C=C, intercept_scaling=intercept_scaling)
    model.fit(train_vectors, y_train)
    y_pred = model.predict(test_vectors)
    return accuracy_score(y_test, y_pred)

In [8]:
#initial population
population = []
for i in range(10):
    population.append( ( random.uniform(0, 5),
                      random.uniform(0, 5),
                      random.uniform(0, 5) ) )
    
temp = ()

In [12]:
#implemention Genetic Algoritm

max_acc = (0, (0, 0, 0))

for it in range(100):
    lkj = 0
    rank = []
    for individual in population:
        acc = fitness(individual[0], individual[1], individual[2])
        rank.append((acc, individual))
    
    rank.sort()
    rank.reverse()
    rank = rank[:5]
    pool = []
    for i in rank:
        pool.append(i[1][0])
        pool.append(i[1][1])
        pool.append(i[1][2])

    new_population = []
    for _ in range(10):
        x = random.choice(pool) * random.uniform(0.9, 1.1)
        y = random.choice(pool) * random.uniform(0.99, 1.01)
        z = random.choice(pool) * random.uniform(0.99, 1.01)
        new_population.append((x, y, z))

    population = new_population
    local_best_fit = rank[0][1]
    
    # if fitness(local_best_fit[0], local_best_fit[1], local_best_fit[2]) >= 90:
    #     print(local_best_fit)
    #     temp = local_best_fit
    #     break
    if rank[0][0] > max_acc[0]:
        max_acc = (rank[0][0], local_best_fit)
    print(it, rank[0][0])

print(max_acc)

0 0.8704
1 0.8698
2 0.8694
3 0.8682
4 0.869
5 0.8706
6 0.868
7 0.8690666666666667
8 0.8677333333333334
9 0.8671333333333333
10 0.8687333333333334
11 0.8692
12 0.8713333333333333
13 0.8724666666666666
14 0.8720666666666667
15 0.8708666666666667
16 0.871
17 0.8708666666666667
18 0.8707333333333334
19 0.8714666666666666
20 0.8728
21 0.8721333333333333
22 0.8732
23 0.8730666666666667
24 0.8716
25 0.8734666666666666
26 0.8704666666666667
27 0.8742
28 0.8718
29 0.8756
30 0.873
31 0.8718666666666667
32 0.8718666666666667
33 0.8738
34 0.8739333333333333
35 0.8728666666666667


In [28]:
fitness(1e-4, 1.0, 1.0)

c:\Users\AYUSH\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


0.8616